In [ ]:
import pandas as pd 
import matplotlib as plt



def loader_validation () : 

    """
    Load historical Nasdaq data, review entries, and plot closing prices for quick data validation.

    Steps
    -----
    - Reads the CSV file from 'data/raw/NQ.csv' into a pandas DataFrame.
    - Displays the first five rows to verify dataset structure and format.
    - Displays the last five rows to check for completeness at the dataset boundaries.
    - Plots the time series of 'Close' prices for visual sanity check of data trends and content.

    Returns
    -------

    Example
    -------
    >>> df = pd.read_csv("data/raw/NQ.csv")
    >>> print(df.head())
    >>> print(df.tail())
    >>> plt.figure(figsize=(10, 4))
    >>> plt.plot(pd.to_datetime(df['Date']), df['Close'])
    >>> plt.xlabel('Date')
    >>> plt.ylabel('Close Price')
    >>> plt.title('Nasdaq (NQ) Close Price Series')
    >>> plt.tight_layout()
    >>> plt.show()
    """

    # Load the data
    df = pd.read_csv("data/raw/NQ.csv")

    # Display the first rows
    print("First rows:")
    print(df.head())

    # Display the last rows
    print("\nLast rows:")
    print(df.tail())

    # Sanity check: plot the price series (here 'Close')
    plt.figure(figsize=(10, 4))
    plt.plot(pd.to_datetime(df['Date']), df['Close'])
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Nasdaq (NQ) Close Price Series')
    plt.tight_layout()
    plt.show()
    
    return df